In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.random.seed(98765)

In [2]:

from sklearn.linear_model import RidgeCV
from tqdm import tqdm

In [3]:
import joblib

In [4]:
dd = pd.read_csv("PRSA_Data_Aotizhongxin_20130301-20170228.csv")
dd.head()

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,1,2013,3,1,0,4.0,4.0,4.0,7.0,300.0,77.0,-0.7,1023.0,-18.8,0.0,NNW,4.4,Aotizhongxin
1,2,2013,3,1,1,8.0,8.0,4.0,7.0,300.0,77.0,-1.1,1023.2,-18.2,0.0,N,4.7,Aotizhongxin
2,3,2013,3,1,2,7.0,7.0,5.0,10.0,300.0,73.0,-1.1,1023.5,-18.2,0.0,NNW,5.6,Aotizhongxin
3,4,2013,3,1,3,6.0,6.0,11.0,11.0,300.0,72.0,-1.4,1024.5,-19.4,0.0,NW,3.1,Aotizhongxin
4,5,2013,3,1,4,3.0,3.0,12.0,12.0,300.0,72.0,-2.0,1025.2,-19.5,0.0,N,2.0,Aotizhongxin


In [17]:
dd[dd["PM2.5"]>20]

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
23,24,2013,3,1,23,24.0,24.0,26.0,54.0,600.0,36.0,-0.2,1030.5,-17.4,0.0,ENE,1.4,Aotizhongxin
24,25,2013,3,2,0,22.0,24.0,24.0,44.0,500.0,44.0,-0.4,1031.0,-17.6,0.0,ENE,1.4,Aotizhongxin
36,37,2013,3,2,12,26.0,30.0,25.0,76.0,900.0,22.0,2.7,1027.3,-16.4,0.0,WSW,2.7,Aotizhongxin
37,38,2013,3,2,13,25.0,33.0,20.0,57.0,700.0,39.0,3.3,1025.7,-16.4,0.0,W,1.5,Aotizhongxin
38,39,2013,3,2,14,26.0,35.0,22.0,54.0,700.0,43.0,3.8,1024.8,-16.0,0.0,SE,1.3,Aotizhongxin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35037,35038,2017,2,27,21,53.0,82.0,9.0,124.0,1200.0,24.0,13.0,1014.7,-8.9,0.0,SSW,2.1,Aotizhongxin
35038,35039,2017,2,27,22,58.0,106.0,17.0,133.0,1500.0,19.0,12.9,1015.3,-8.3,0.0,NNW,2.1,Aotizhongxin
35039,35040,2017,2,27,23,70.0,107.0,28.0,131.0,1700.0,19.0,10.9,1015.6,-7.4,0.0,NNW,1.4,Aotizhongxin
35040,35041,2017,2,28,0,56.0,56.0,22.0,119.0,1400.0,18.0,9.9,1015.6,-8.3,0.0,NNE,1.5,Aotizhongxin


In [5]:
def select_data_path(category):
    data_paths = {
        'Aotizhongxin': 'path/to/data_A.csv',
        'Changping': 'path/to/data_B.csv',
        'Dingling': 'path/to/data_C.csv'
    }
    return data_paths.get(category, 'Invalid category')


In [6]:
def load_bejing_air_data(data_path):
    data = pd.read_csv(data_path)
    data.drop(columns=['No', 'station'], inplace=True, axis=1)
    data.drop(columns=['year', 'month', 'day', 'hour'], inplace=True, axis=1)
    data_wd = data['wd'].fillna(value="Unknown")
    data = data.loc[:, data.columns != 'wd'].fillna(method="ffill")
    data = data.fillna(method="bfill")
    data['wd'] = data_wd
    Y_full = data['PM2.5']
    data.drop(columns=['PM2.5', 'PM10'], inplace=True, axis=1)
    data['wd_h'] = data['wd'].apply(lambda x: _encode_direction(x, True))
    data['wd_v'] = data['wd'].apply(lambda x: _encode_direction(x, False))
    data.drop(columns=["wd"], inplace=True, axis=1)
    X_full = data
    return X_full.to_numpy().astype('float64'),Y_full.to_numpy().astype('float64'),X_full,Y_full

def _encode_direction(direction, horizontal):
    if horizontal:
        if direction in ['N', 'S']:
            return 0
        elif direction in ['NNW', 'SSW']:
            return -0.5
        elif direction in ['NW', 'SW']:
            return -0.7
        elif direction in ['WNW', 'WSW']:
            return -0.86
        elif direction == 'W':
            return -1
        elif direction in ['NNE', 'SSE']:
            return 0.5
        elif direction in ['NE', 'SE']:
            return 0.7
        elif direction in ['ENE', 'ESE']:
            return 0.86
        elif direction == 'E':
            return 1
        elif direction == 'Unknown':
            return 0
        else:
            raise ValueError("Invalid Dir")
    else:
        if direction in ['W', 'E']:
            return 0
        elif direction in ['WSW', 'ESE']:
            return -0.5
        elif direction in ['SW', 'SE']:
            return -0.7
        elif direction in ['SSW', 'SSE']:
            return -0.86
        elif direction == 'S':
            return -1
        elif direction in ['WNW', 'ENE']:
            return 0.5
        elif direction in ['NW', 'NE']:
            return 0.7
        elif direction in ['NNW', 'NNE']:
            return 0.86
        elif direction == 'N':
            return 1
        elif direction == 'Unknown':
            return 0
        else:
            raise ValueError("Invalid Dir")



In [7]:
def norma(X,Y):
    X_mean = np.mean(X, axis=0)
    X_std = np.std(X, axis=0)
    X_normalized = (X - X_mean) / X_std
    Y_mean = np.mean(Y, axis=0)
    Y_std = np.std(Y, axis=0)
    Y_normalized = (Y - Y_mean) / Y_std
    return X_normalized,Y_normalized, X_mean, X_std, Y_mean, Y_std


In [8]:
X, Y,X_1,Y_1 = load_bejing_air_data('PRSA_Data_Aotizhongxin_20130301-20170228.csv')
Y = np.log(Y)
X, Y, X_mean, X_std, Y_mean, Y_std = norma(X, Y)

/var/folders/cm/4r8lp84x5jqfnzlpt29yt9dr0000gn/T/ipykernel_1290/1754808785.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.loc[:, data.columns != 'wd'].fillna(method="ffill")
/var/folders/cm/4r8lp84x5jqfnzlpt29yt9dr0000gn/T/ipykernel_1290/1754808785.py:7: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method="bfill")


In [10]:
type(X)

numpy.ndarray

In [13]:
np.save('X_data.npy', X)
np.save('X_mean.npy', X_mean)
np.save('X_std.npy', X_std)
np.save('Y_data.npy', Y)
np.save('Y_mean.npy', Y_mean)
np.save('Y_std.npy', Y_std)

In [14]:
def modelling(X,Y):
    n = len(Y)
    # odd data points for training, even for calibration
    inds_odd = np.arange(1,int(np.ceil(n/2)*2-1),2)
    inds_even = np.arange(2,int(np.floor(n/2)*2),2)

    X_train = X[inds_odd]
    y_train = Y[inds_odd]
    X_calib = X[inds_even]
    y_calib = Y[inds_even]

# # train model
    min_alpha = 0.0001
    max_alpha = 10
    model = RidgeCV(alphas=np.linspace(min_alpha, max_alpha, 10))
    model = RidgeCV(alphas=np.linspace(min_alpha, max_alpha, 500))
    model.fit(X_train, y_train)
    model.alpha_
    return X, Y, model

In [15]:
X, Y, model = modelling(X,Y)

In [16]:
joblib.dump(model, 'ridge_model.joblib')

['ridge_model.joblib']

In [ ]:
def denormalize(Y_normalized, Y_mean, Y_std):
    Y_original = Y_normalized * Y_std + Y_mean
    return  Y_original

In [ ]:
locations_1 = {
    "New York": "40.712776,-74.005974",
    "Los Angeles": "34.052235,-118.243683",
    "Chicago": "41.878113,-87.629799",
    "Houston": "29.760427,-95.369804",
    "Phoenix": "33.448376,-112.074036",
    "Philadelphia": "39.952583,-75.165222",
    "San Antonio": "29.424349,-98.491142",
    "San Diego": "32.715736,-117.161087",
    "Dallas": "32.776665,-96.796989"
}

In [ ]:
def pm_to_aqi(aqi_ranges, concentration_range):
    start_conc, end_conc = concentration_range
    largest_index = None
    largest_start = -1  # Track the largest starting point of overlapping ranges

    for idx, aqi_range in enumerate(aqi_ranges):
        aqi_start, aqi_end = aqi_range
        # Check for overlap
        if aqi_end == float('inf'):
            if start_conc >= aqi_start:
                # Infinite overlap size
                if largest_index is None or aqi_start > largest_start:
                    largest_index = idx
                    largest_start = aqi_start
        elif start_conc < aqi_end and end_conc > aqi_start:
            # Calculate overlap
            if aqi_start > largest_start:
                largest_index = idx
                largest_start = aqi_start
    
    return largest_index  # Return the index of the range with the highest starting point or None if no overlap found






# Example AQI ranges and a concentration range
aqi_ranges = [(0, 12), (12.1, 35.4), (35.5, 55.4),(55.5 ,150.4),(150.5,250.4),(250.5,float('inf'))]



def get_outputs(index):
    if index == 0:
        level = "Good"
        color = "lightgreen"
        aq = "Air quality is satisfactory, and air pollution poses little or no risk."
        coe = "No specific actions are needed. Enjoy normal outdoor activities."
    elif index == 1:
        level = "Moderate"
        color = "yellow"
        aq = "AIr quality is acceptable; however, some pollutants may be a concern for a very small number of people."
        coe = "People who are unusually sensitive to air pollution may consider limiting prolonged outdoor exertion."
    elif index == 2:
        level = "Unhealthy for sensitive people"
        color = "orange"
        aq ="Health effects may start to impact sensitive groups."
        coe = "Sensitive groups (children, elderly, people with respiratory issues) should limit prolonged outdoor exertion. Everyone else can continue outdoor activities but should remain cautious."
    elif index == 3:
        level = "Unhealthy"
        color = "red"
        aq = "Health effects may be experienced by everyone, with sensitive individuals at greater risk."
        coe = "Everyone should reduce prolonged outdoor exertion. Sensitive individuals should avoid outdoor activities. Indoor air purifiers can be beneficial."
    elif index == 4:
        level = "Very unhealthy"
        color = "darkred"
        aq = "Health warnings of emergency conditions; the entire population may experience health effects."
        coe = "Avoid outdoor activities if possible. All individuals, especially sensitive groups, should remain indoors and use air purifiers. Wear N95 masks if outdoor exposure is necessary."
    elif index == 5:
        level = "Hazardous"
        color = "Magenta"
        aq = "Serious health effects likely for the entire population."
        coe = " Everyone should stay indoors and keep windows and doors closed. Avoid physical activity and consider relocating temporarily if exposure is prolonged. Use air purifiers and tightly seal indoors."
    
    return level, color,aq, coe

In [ ]:
import gradio as gr
rho = 0.99; rho_LS = 0.99
alpha = 0.1
locations = {
        'Aotizhongxin': '39.989444, 116.409722',
        'Changping': '40.218086, 116.235908',
        'Dingling': '40.290168, 116.220278'
    }

# Function to process the inputs and return a numerical output
def process_inputs(num1, num2, num3, num4, num5, num6, num7, num8, num9, wind, location):
    # Example logic: sum the numerical inputs and assign a multiplier based on the categorical input
    wind_horizontal = {
        "N": 0, "S":0,
        "NNW": -0.5,"SSE":-0.5,
        "SW":-0.7,"NW": -0.7,
        "WNW": -0.86,"WSW": -0.86,
        "W": -1,
        "NNE": 0.5, "SSE":0.5,
        "NE": 0.7, "SE":0.7,
        "ENE": 0.86,
        "ESE":0.86,
        "E": 1,
        "Unknown": 0
    }
    wind_vertical = {
        "W": 0, "E":0,
        "WSW": -0.5,"ESE":-0.5,
        "SW":-0.7,"SE": -0.7,
        "SSW": -0.86,"SSE": -0.86,
        "S": -1,
        "WNW": 0.5, "ENE":0.5,
        "NW": 0.7, "NE":0.7,
        "NNW": 0.86,
        "NNE":0.86,
        "N": 1,
        "Unknown": 0
    }
    w_h = wind_horizontal.get(wind, 0)
    w_v = wind_vertical.get(wind, 0)

    data = {
    "S02": [num1],"N02": [num2],"CO": [num3],"O3": [num4],"Temp": [num5],"Pres": [num6],
    "DW": [num7],"Rain": [num8],"WSPM": [num9], "wind_h":[w_h], "wind_v":[w_v]
    
    }

    # Create DataFrame from dictionary
    df = pd.DataFrame(data)
    
    X_test = df.to_numpy().astype('float64')
    X_norma = (X_test - X_mean) / X_std


    n = len(Y)
    weights=rho**(np.arange(n,0,-1))
    tags=rho_LS**(np.arange(n,-1,-1))
    loss_beta = 1.0 #loss upper bound
    lambdas = np.arange(0,2,0.01) #set lambdas range (lambda*2)

    # odd data points for training, even for calibration
    inds_odd = np.arange(1,int(np.ceil(n/2)*2-1),2)
    inds_even = np.arange(2,int(np.floor(n/2)*2),2)

    X_calib = X[inds_even]
    y_calib = Y[inds_even]


    # get predictions on test set
    Y_pred = model.predict(X_calib)

    # compute residuals
    residuals = np.abs(y_calib - Y_pred)

    # compute weighted empirical risk in the calibration set
    losses = np.zeros((len(lambdas), len(residuals)))
    n_w = np.sum(weights[inds_even] )
    r_hats = np.zeros(len(lambdas))

    for li,l in enumerate(lambdas):
        losses[li,:]=np.maximum(np.zeros((residuals.shape)),residuals-(l/2))
        ws = np.sum(weights[inds_even]*losses[li])
        r_hats[li] = (1/n_w) * ws

    calib_lambdas = (r_hats*n_w/(n_w+1)) + loss_beta/(n_w+1)
    lambda_chosen = np.max(lambdas)

    # find the infimum of lambdas
    for i,li in enumerate(calib_lambdas):
        if li<=alpha:
            lambda_chosen = lambdas[i]
            break


    x_1 = X_norma.reshape(1, -1)
    y_PI = np.array([denormalize(model.predict(x_1), Y_mean ,Y_std)-(lambda_chosen/2),denormalize(model.predict(x_1), Y_mean ,Y_std)+(lambda_chosen/2)])
    

    # Get coordinates for selected location
    coordinates = locations.get(location, "0,0")  # Default to "0,0" if location not found
    
    lat, lng = coordinates.split(",")
    
    # Generate the OpenStreetMap iframe embed code with the selected coordinates
    osm_map_html = f"""
    <iframe 
      width="100%" 
      height="400" 
      frameborder="0" 
      scrolling="no" 
      marginheight="0" 
      marginwidth="0" 
      src="https://www.openstreetmap.org/export/embed.html?bbox={float(lng)-0.05}%2C{float(lat)-0.05}%2C{float(lng)+0.05}%2C{float(lat)+0.05}&layer=mapnik&marker={lat}%2C{lng}" 
      style="border: 1px solid black"></iframe>
    """

    result = pm_to_aqi(aqi_ranges, (np.exp(y_PI[0][0]),np.exp(y_PI[1][0])))

    level, color,aq, coe = get_outputs(result)
    
    out_2_html = f"<span style='color: {color}; font-size: 20px; text-transform: uppercase;'>{level}</span>"


    return str(round(np.exp(float(y_PI[0][0])),2)) + " to " + str(round(np.exp(float(y_PI[1][0])),2)), osm_map_html, out_2_html, aq, coe
    
# Info dictionary for each parameter
parameter_info = {
    "SO2": "Sulfur Dioxide (SO2) is a toxic gas produced by volcanic activity and industrial processes, notably the combustion of fossil fuels. Measured in micrograms per cubic meter (µg/m³). Typical Range: 0 to 100 µg/m",
    "NO2": "Nitrogen Dioxide (NO2) is a reddish-brown gas with a characteristic sharp, biting odor, a significant air pollutant from combustion processes. Measured in micrograms per cubic meter (µg/m³). Typical Range: 0 to 200 µg/m",
    "CO": "Carbon Monoxide (CO) is a colorless, odorless gas that can be harmful when inhaled in large amounts, often produced by incomplete combustion. Measured in micrograms per cubic meter (µg/m³). Typical Range: 0 to 10000 µg/m",
    "O3": "Ozone (O3) at ground level is an air pollutant that affects lung function and is formed from reactions between sunlight and other pollutants. Measured in micrograms per cubic meter (µg/m³). Typical Range: 0 to 180 µg/m",
    "Temperature": "Temperature in meteorology is a measure of the warmth or coldness of the environment. Measured in Degrees Celsius (°C). Typical Range: -20 to 40 °C; varies with seasons.",
    "Pressure": "Pressure refers to the atmospheric force exerted on a surface, essential in predicting weather patterns. Measured in hectopascals (hPa). Typical Range: 900 to 1050 hPa generally lower in summer and higher in winter.",
    "Dew Point": "The Dew Point is the temperature at which air becomes saturated with moisture, crucial for humidity measurement. Measured in degrees Celsius (°C) Typical Range: -20 to 30 °C; higher values indicate more humidity.",
    "Rain": "Rainfall measurement indicates the volume of precipitation,  measured in Millimeters (mm) .Typical Range: 0 to 300 mm; varies significantly by season and weather patterns.",
    "WSPM": "Wind Speed per Minute (WSPM) refers to the rate at which air moves past a certain point. Measured in meters per second (m/s). Typical Range: 0 to 10 m/s; can be higher during storms.",
    "Wind Direction": "Wind Direction indicates the direction from which the wind originates, essential for weather predictions. eg: NNW denotes North-Northwest",
    "Location": "Location selection allows you to specify the area where the air quality parameters are being recorded or predicted."
}

# Function to return info text based on the parameter selected
def show_info(parameter):
    return parameter_info.get(parameter, "Information not available.")


# Define the Gradio interface
with gr.Blocks() as demo:
    # Center-aligned heading
    gr.Markdown("<h2 style='text-align: center;'>PM2.5 Air Quality Prediction </h2>")
    gr.Markdown("<p style='text-align: center;'>Enter air quality metrics and weather parameters to predict PM2.5 concentration, AQI level and suggested course of action.</p>")
    # Arrange inputs in two columns
    with gr.Row():
        with gr.Column():
    # Create the input components
            num1 = gr.Textbox(label="SO2", value=4.0)
            gr.Button("Info").click(lambda: show_info("SO2"), None, gr.Markdown(), show_progress=False)
            num2 = gr.Textbox(label="NO2", value=7.0)
            gr.Button("Info").click(lambda: show_info("NO2"), None, gr.Markdown(), show_progress=False)
            
            num3 = gr.Textbox(label="CO", value=300.0)
            gr.Button("Info").click(lambda: show_info("CO"), None, gr.Markdown(), show_progress=False)
            
            num4 = gr.Textbox(label="O3", value=77.0)
            gr.Button("Info").click(lambda: show_info("O3"), None, gr.Markdown(), show_progress=False)
            
            num5 = gr.Textbox(label="Temperature", value=-0.7)
            gr.Button("Info").click(lambda: show_info("Temperature"), None, gr.Markdown(), show_progress=False)
        with gr.Column():
            num6 = gr.Textbox(label="Pressure", value=1023.0)
            gr.Button("Info").click(lambda: show_info("Pressure"), None, gr.Markdown(), show_progress=False)
            
            num7 = gr.Textbox(label="Dew point", value=-18.8)
            gr.Button("Info").click(lambda: show_info("Dew Point"), None, gr.Markdown(), show_progress=False)
            
            num8 = gr.Textbox(label="Rain", value=0.0)
            gr.Button("Info").click(lambda: show_info("Rain"), None, gr.Markdown(), show_progress=False)
            
            num9 = gr.Textbox(label="WSPM", value=4.4)
            gr.Button("Info").click(lambda: show_info("WSPM"), None, gr.Markdown(), show_progress=False)
            
            category1 = gr.Dropdown(
                choices=["N", "S", "NNW", "SSE", "SW", "NW", "WNW", "WSW", "W", "NNE", "SSE",
                         "NE", "SE", "ENE", "ESE", "E", "Unknown"],
                label="Wind Direction", 
                value="NNW"
            )
            gr.Button("Info").click(lambda: show_info("Wind Direction"), None, gr.Markdown(), show_progress=False)
    
    
    
    location = gr.Dropdown(choices=list(locations.keys()), label="Select a Location")
    gr.Button("Info").click(lambda: show_info("Location"), None, gr.Markdown(), show_progress=False)
    
    
    # Map embed using iframe in Gradio
    location_html = gr.HTML(label="Selected Location on Map")
    
    # Create the output component
    output_1 = gr.Textbox(label="PM2.5 concentration range in µg/m", interactive=False)
    
    gr.Markdown("### AQI level")
    output_html= gr.HTML(label="AQI scale")
    
    out_aq = gr.Textbox(label="Assosiated Health risk", interactive=False)
    out_coe = gr.Textbox(label="Suggested course of action", interactive=False)
    # Create the submit button and link to the function
    submit_button = gr.Button("Submit")
    
    submit_button.click(process_inputs, inputs=[num1, num2, num3, num4, num5, num6, num7, num8, num9, category1, location], outputs=[output_1,location_html, output_html, out_aq, out_coe])

# Launch the Gradio interface
demo.launch()


In [ ]:
xx = [[1.94336546],
 [2.88336546]]

In [ ]:
xx[1][0]

2.88336546

In [ ]:
fffff

NameError: name 'fffff' is not defined

[[1.94336546]
 [2.88336546]]


In [ ]:
import gradio as gr

# List of fixed locations with their coordinates (latitude and longitude)
locations = {
    "New York": "40.712776,-74.005974",
    "Los Angeles": "34.052235,-118.243683",
    "Chicago": "41.878113,-87.629799",
    "Houston": "29.760427,-95.369804",
    "Phoenix": "33.448376,-112.074036",
    "Philadelphia": "39.952583,-75.165222",
    "San Antonio": "29.424349,-98.491142",
    "San Diego": "32.715736,-117.161087",
    "Dallas": "32.776665,-96.796989"
}

# Function to process inputs and return a map iframe URL based on selected location
def process_inputs(num1, num2, num3, num4, category, location):
    category_multiplier = {
        "Category A": 1.1,
        "Category B": 1.2,
        "Category C": 1.3
    }
    multiplier = category_multiplier.get(category, 1.0)
    
    # Sum of the numerical inputs with a multiplier from category
    result = (num1 + num2 + num3 + num4) * multiplier
    
    # Get coordinates for selected location
    coordinates = locations.get(location, "0,0")  # Default to "0,0" if location not found
    
    lat, lng = coordinates.split(",")
    
    # Generate the OpenStreetMap iframe embed code with the selected coordinates
    osm_map_html = f"""
    <iframe 
      width="100%" 
      height="400" 
      frameborder="0" 
      scrolling="no" 
      marginheight="0" 
      marginwidth="0" 
      src="https://www.openstreetmap.org/export/embed.html?bbox={float(lng)-0.05}%2C{float(lat)-0.05}%2C{float(lng)+0.05}%2C{float(lat)+0.05}&layer=mapnik&marker={lat}%2C{lng}" 
      style="border: 1px solid black"></iframe>
    """
    
    return result, osm_map_html

# Gradio interface
with gr.Blocks() as demo:
    num1 = gr.Number(label="Numerical Input 1")
    num2 = gr.Number(label="Numerical Input 2")
    num3 = gr.Number(label="Numerical Input 3")
    num4 = gr.Number(label="Numerical Input 4")
    category = gr.Dropdown(choices=["Category A", "Category B", "Category C"], label="Categorical Input")
    
    # Dropdown for location selection
    location = gr.Dropdown(choices=list(locations.keys()), label="Select a Location")
    
    # Map embed using iframe in Gradio
    location_html = gr.HTML(label="Selected Location on Map")
    
    output_num = gr.Number(label="Numerical Output")
    
    submit_button = gr.Button("Submit")
    
    submit_button.click(process_inputs, 
                        inputs=[num1, num2, num3, num4, category, location], 
                        outputs=[output_num, location_html])

demo.launch()


In [ ]:
split into pollutants and weather parameters
add data path according to location
correct interval logic: check

mealnona: diagnosis text of each class, last say see doctor
if pm at threshold value

In [ ]:
bbox: Defines the map's boundaries (left, bottom, right, top), which we calculate based on the location's latitude and longitude.
marker: This places a pin at the selected location.
Adjustments:
You can change the zoom level by adjusting the bounding box (bbox). The current setup zooms in to a reasonable level for city views.
You can also customize the size of the map by adjusting the width and height of the iframe.

In [ ]:
pm_to_aqi(aqi_ranges, (1,2))

5